In [13]:
import os, sys
import numpy as np
import json
import neuroglancer
import pandas as pd
import ast
from matplotlib import pyplot as plt
viewer = neuroglancer.Viewer()
print(viewer)

http://127.0.0.1:44561/v/bb5cb94b1dec4a01298720f5cba436da590b3b9f/


In [7]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)

from utilities.contour_utilities import get_contours_from_annotations, add_structure_to_neuroglancer, \
    create_full_volume, get_structure_colors
from utilities.imported_atlas_utilities import get_all_structures, get_structures

Connecting dklab@192.168.1.12:3306


In [14]:
xy_ng_resolution_um = 5
color_radius = 3
animal = 'MD594'

CSV_PATH = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/foundation_brain_annotations'
csvfile = os.path.join(CSV_PATH, f'{animal}_annotation.csv')
hand_annotations = pd.read_csv(csvfile)
hand_annotations['vertices'] = hand_annotations['vertices'] \
    .apply(lambda x: x.replace(' ', ','))\
    .apply(lambda x: x.replace('\n',','))\
    .apply(lambda x: x.replace(',]',']'))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ',')).apply(lambda x: x.replace(',,', ','))

hand_annotations['vertices'] = hand_annotations['vertices'].apply(lambda x: ast.literal_eval(x))

In [15]:
structures = get_structures()
#
structures_arr = hand_annotations.name.unique()
annotation_structures = structures_arr.tolist()
structures = [a for a in annotation_structures if a in structures]
colors = get_structure_colors()

In [17]:
structures

['SC', 'IC', '12N', 'AP', 'RtTg']

In [ ]:
color_filepath = os.path.join('../', 'neuroglancer/contours/json_cache', 'struct_to_color_2.json')
with open(color_filepath, 'r') as json_file:
    colors = json.load(json_file)
colors = {name.upper(): index for name, index in colors.items()}
    
surround = False
VOL_DIR = '/net/birdstore/Active_Atlas_Data/copied_from_S3/mousebrainatlas-data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes'
files = os.listdir(VOL_DIR)
volume_files = sorted([f for f in files if f.endswith('.npy') and surround == ('surround' in f) and 'test' not in f])
origin_files = sorted([f for f in files if f.endswith('.txt') and surround == ('surround' in f) and 'test' not in f])
    
structure_volume_origin = {}
for volume_filename, origin_filename in zip(volume_files, origin_files):
    prefix = os.path.splitext(volume_filename)[0]
    structure = prefix.replace('atlasV7_10.0um_scoreVolume_', '').replace('_surround_200um', '')
    if structure not in origin_filename:
        print(structure, origin_filename)
        break
    
    try:
        color = colors[structure.upper()]
    except:
        sided = '{}_R'.format(structure.upper())
        color = colors[sided]

    volume = np.load(os.path.join(VOL_DIR, volume_filename))
    origin = np.loadtxt(os.path.join(VOL_DIR, origin_filename))
    
    volume = np.rot90(volume, axes=(0,1))
    volume = np.flip(volume, axis=0)
    volume[volume > 0.8] = color
    volume = volume.astype(np.uint8)
    
    structure_volume_origin[structure] = (volume, origin)
print(structure_volume_origin.keys())

In [22]:
structure_volume_origin = {}

for structure in structures:
    try:
        color = colors[structure.upper()]
    except:
        sided = '{}_R'.format(structure)
        try:
            color = colors[sided]
        except:
            color = 100

    print(structure, color, end="\t")
    contour_annotations, first_sec, last_sec = get_contours_from_annotations(animal, structure, hand_annotations,
                                                                             densify=4)
    if first_sec == 0 or last_sec == 0:
        print('No sections found')
        continue
    else:
        print('Section start, end:', first_sec, last_sec)

    threshold = 1
    volume, xyz_offsets = create_full_volume(contour_annotations, structure, first_sec, last_sec, \
                                                       color_radius, xy_ng_resolution_um, threshold, color)
    structure_volume_origin[structure] = (volume, xyz_offsets)

SC 18	Section start, end: 128 311
IC 11	Section start, end: 122 314
12N 2	Section start, end: 206 245
AP 9	Section start, end: 215 230
RtTg 17	Section start, end: 200 275


In [23]:
x_length = 1000
y_length = 1000
z_length = 300
full_brain_volume_annotated = np.zeros((x_length, y_length, z_length), dtype=np.uint32)

for structure, (volume, origin) in structure_volume_origin.items():  
    
    x, y, z = origin
    x_start = int(x) + x_length // 2
    y_start = int(y) + y_length // 2
    z_start = int(z) // 2 + z_length // 2
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = z_start + (volume.shape[2] + 1) // 2
    
    print(structure,'X range', x_start, x_end, end="\t")
    print('Y range', y_start, y_end, end="\t")
    print('Z range', z_start, z_end)
    continue


    z_indices = [z for z in range(volume.shape[2]) if z % 2 == 0]
    volume = volume[:, :, z_indices]
    #full_brain_volume_annotated[x_start:x_end, y_start:y_end,z_start:z_end] += volume


all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=full_brain_volume_annotated, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[10, 10, 20]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

SC X range 1966 2185	Y range 910 2077	Z range 168 1411
IC X range 2007 2229	Y range 915 2185	Z range 165 1466
12N X range 2976 3129	Y range 1595 3138	Z range 207 1681
AP X range 3144 3282	Y range 1585 2978	Z range 211 1656
RtTg X range 2252 2435	Y range 1663 3220	Z range 204 1358


In [ ]:
import tinybrain
from cloudvolume import CloudVolume

path = 'file:///net/birdstore/Active_Atlas_Data/data_root/pipeline_data/test'
factor = (2, 2, 1)
volumes = tinybrain.downsample_segmentation(full_brain_volume_annotated, factor=factor, num_mips=2, sparse=False)
volumes.insert(0, full_brain_volume_annotated)

In [ ]:
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = 'segmentation',
    data_type = 'uint32', # Channel images might be 'uint8'
    encoding = 'compressed_segmentation', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = [10000, 10000, 20000], # Voxel scaling, units are in nanometers
    voxel_offset = [0, 0, 0], # x,y,z offset in voxels from the origin
    chunk_size = [512, 512, 16], # units are voxels
    volume_size = full_brain_volume_annotated.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(path, info=info, compress=False, progress=False)

for mip, volume in enumerate(volumes):
    vol.add_scale(np.array(factor) ** mip)
    vol.commit_info()
    vol = CloudVolume(path, mip=mip, compress=False, progress=False)
    vol[:, :, :] = volume[:, :, :]
    
vol.info['segment_properties'] = 'names'
vol.commit_info()